In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import pandas as pd
from tqdm import tqdm

In [2]:
MODEL_NAME = "IlyaGusev/saiga2_13b_lora"
BASE_MODEL_PATH = "TheBloke/Llama-2-13B-fp16"
# BASE_MODEL_PATH = "meta-llama/Llama-2-13b-hf"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]
        self.system_prompt=system_prompt

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, texts, tokenizer):
        final_texts = []
        for text_ in texts:
            text = text_[2]
            self.messages = [{
                "role": "system",
                "content": self.system_prompt
            }]

            self.add_user_message(text)
            
            final_text = ""
            for message in self.messages:
                message_text = self.message_template.format(**message)
                final_text += message_text
                
            final_text += tokenizer.decode([self.start_token_id, self.bot_token_id]).strip()
            final_texts.append(final_text)

        return final_texts


In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

def model_answer(input):
    conversation = Conversation()
    conversation.add_user_message(input)
    prompt = conversation.get_prompt(tokenizer)
    model_output = generate(model, tokenizer, prompt, generation_config)
    torch.cuda.empty_cache()
    return model_output

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.15,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [4]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", padding=True)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )
    output_ids = [output[len(data["input_ids"][0]):] for output in output_ids]
    output = [tokenizer.decode(out, skip_special_tokens=True) for out in output_ids]
    return output

In [5]:
#%%time
#output = generate(model, tokenizer, ["2+2", "3+5"], generation_config)

In [6]:
df = pd.read_csv("data/train.csv")
bad_themes = ["Государственная собственность", "Культура", 'МФЦ "Мои документы"', "Памятники и объекты культурного наследия"
             "Погребение и похоронное дело", "Роспотребнадзор", "Спецпроекты", "Строительство и архитектура", "Торговля",
              "Физическая культура и спорт", "Экология", "Экономика и бизнес", "Электроснабжение"]

In [7]:
prompt = "[ИНФОРМАЦИЯ]:\nПользовательские обращения:\n {}\nТема: {}\n[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.\nОтвет:"

In [8]:

batch_size = 2

In [9]:
list_data = []
for theme in tqdm(bad_themes):
    for pod_theme in df[df["Группа тем"] == theme].Тема.unique().tolist():
            for ispolnitel in df[(df["Группа тем"] == theme) & (df["Тема"] ==pod_theme)]["Исполнитель"].tolist()[:3]:
                df_test = df[(df["Группа тем"] == theme) & (df["Тема"] ==pod_theme) & (df["Исполнитель"] == ispolnitel)]
                incedents = "\n".join([f"{i+1}. {example.strip()}" for i, example in enumerate(df_test["Текст инцидента"][:3].to_list())])
                prompt_ = prompt.format(incedents, pod_theme)
                list_data.append([ispolnitel, theme, prompt_, pod_theme])

100%|██████████| 12/12 [00:00<00:00, 13.28it/s]


In [10]:
batches = [list_data[i:i + batch_size] for i in range(0, len(list_data), batch_size)]

In [11]:
len(batches)

33

In [12]:
# 9

In [13]:
results = []

In [14]:
for batch in tqdm(batches[9:]):
    conversation = Conversation()
    prompt_ = conversation.get_prompt(batch, tokenizer)
    output = generate(model, tokenizer, prompt_, generation_config)
    
    for b, o in zip(batch, output):
        print(b[2]+"\n")
        print('____')
        print(o)
        b[2] = o
        print('---------------\n')
        print(b)
        with open("result_.txt", 'a') as file:
            file.write(str(b)+ '\n')

    torch.cuda.empty_cache()

  4%|▍         | 1/24 [04:35<1:45:26, 275.05s/it]

[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 1. Пермь г, 79125947946. Здравствуйте. 
Около дома Р. Землячки 8, ближе к ул. Лебедева расплодились крысы. УК Техкомфорт, которая отвечает за данную территорию утверждает что проводит дератацию. Возможно это так, но видимо она не срабатывает. 
Крысеныши бегают вокруг подъезда. 
Подскажите, как действовать, чтобы решить проблему.
2. У нас такая же проблема
3. Вчера проходила возле мусорного контейнера у стахановская 15. Во неописуемо дающий крысы бегают. Везде мусора, горы, особенно около Магазины, пятёрочки.
Тема: Обработка и уничтожение грызунов (дератизация)
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.
Ответ:

____

1. Привет! Хочу сообщить о проблемах с крысами в 

  8%|▊         | 2/24 [06:58<1:12:31, 197.80s/it]

[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 1. Пермь г, 79125947946. Здравствуйте. 
Около дома Р. Землячки 8, ближе к ул. Лебедева расплодились крысы. УК Техкомфорт, которая отвечает за данную территорию утверждает что проводит дератацию. Возможно это так, но видимо она не срабатывает. 
Крысеныши бегают вокруг подъезда. 
Подскажите, как действовать, чтобы решить проблему.
2. У нас такая же проблема
3. Вчера проходила возле мусорного контейнера у стахановская 15. Во неописуемо дающий крысы бегают. Везде мусора, горы, особенно около Магазины, пятёрочки.
Тема: Обработка и уничтожение грызунов (дератизация)
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.
Ответ:

____

1. Привет! Хочу сообщить о проблеме с крысами ряд

 12%|█▎        | 3/24 [10:41<1:13:17, 209.39s/it]

...

[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 1. Да очень много добавили 73рубля куда бы их девать,а пенсия 8600 в личном кабинете стоит сумма 10441.
2. Это смешной прожиточный минимум для пенсионеров,даже унизительный,как будьто пенсионеры самые богатые люди не учитывают сколько идет на лекарство ,а сколько это лекарство стоит мы ходим в те же магазины ,платим за ЖКХ так же ,пенсионеры у нас не впочете, а очень жаль,
3. Можно узнать когда у нас цены на сахар крупы туалет. бум понижение цен будет или ни когда как жить с пенсией 10780.26р.
Тема: Цены и ценообразование
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.
Ответ:

____

1. Не могу поверить, что такие высокие цены на сахар! Я платил уже 50 рублей за килограм

 92%|█████████▏| 22/24 [55:38<03:33, 106.73s/it]

[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 1. Возвращаясь к теме отсутствия электроэнергии 31.07.22 информирую,что ответа на жалобу в Пермэнерго нет.Не считают нужным …
2. Березники г, 79124811750. Добрый день. В посёлке Пешково Усольского района города Березники у жителей домов на улицах Залесье и Полевая напряжение в сети составляет порядка 160 170 Вольт. Обращались коллективно в ПАО Пермэнергосбыт и Березниковские электрические сети ОАО МРСК Урала. Обратной связи не получили, никаких действий данные организации не принимают. Просим помочь.
3. ФЕДОРКИ.Может уже хватит играть светом.Вторая неделя пошла.Никакие электроприборы не работают.Свет мигает что утром,что днем,что вечером до 23.00.
Тема: Качество электроснабжения
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы 

100%|██████████| 24/24 [59:59<00:00, 149.99s/it]

[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 1. На улице Интернациональная с.Барда провис кабель через проезжую часть, после ремонта освещения на опоре ЛЭП.
2. Около частного дома находится столб освещения в аварийном состоянии дерево сгнило и слегка наклонился в сторону дома. Страшно, вдруг упадёт в наш дом.. Или может есть нормы для допустимых наклонов. Кто должен произвести замену столба? Подскажите, куда нужно обращаться или позвонить куда, напишите номера
3. На против пожарный
Тема: ★ Повреждение опор ЛЭП
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от представленных и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.
Ответ:

____

1. На улице Интернациональной с.Барда недавно заметили, что кабель через проезжую часть пролегает под опорой ЛЭП. Это вызывает беспокойство у мес

In [16]:
for o in output:
    print(o)

system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. 
 user
[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 0. Здравствуйте. Кто собственник бывшего училища 6 на ул.Революции? Средь бела дня ломают здание, выбивают стёкла, выносят металл. Или здание демонтируется собственником?
1. Здравствуйте подскажите пожалуйста куда можно обратиться по поводу аренды свободного земельного участка ?
2. Уважаемый губернатор.Перед вашей поездкой в г Лысьва, по вашей просьбе, я сообщил вам о скандале в городе связанном с захватом площади Революции попом Сафонием при помощи администрации, против воли жителей города и депутатов.Однако ответа на свой вопрос не получил.Прошу ответить.
3. Лысьва г, 79051413000. В посёлке кин завод рядом с ним хочу построить кемпинг на 20 домиков круглогодичного пребывания. Куда обратиться за поддержкой по выделению участка ? Спасибо.
4. , Я просил вас отдать продать мне землю, облагародить ее в ответ получил что пока стоит дом, ни 

In [18]:
print(o)

system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. 
 user
[ИНФОРМАЦИЯ]:
Пользовательские обращения:
 0. Добрый час. У меня такой вопрос... В скором времени Губаха и Гремячинский обьеденятся значит рядом находящиеся посёлки Шумихинский юбилейный которые числятся Гремячинский район то тогда уже будет Губахинский район.??? И ещё очень многих интересует а будет ли переселение в Губаху из поселка...??? Будет ли возможность получить жильё в Губахе.???. И если да то какие условия должны соответствовать требованиям... Не все могут позволить купить жильё посёлки практически без работы. Если можно то поделитесь всей информацией на сегодняшний день. Елена
1. По какой причине проводятся кадастровые работы?
Тема: Региональное имущество
[ЗАДАНИЕ]: Опираясь на [ИНФОРМАЦИЯ] пожалуйста, cгенерируй пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным, отличаться от п

In [ ]:
df = pd.read_csv("train_dataset_train_variant2.csv")
bad_themes = ["Государственная собственность", "Культура", 'МФЦ "Мои документы"', "Памятники и объекты культурного наследия"
             "Погребение и похоронное дело", "Роспотребнадзор", "Спецпроекты", "Строительство и архитектура", "Торговля",
              "Физическая культура и спорт", "Экология", "Экономика и бизнес", "Электроснабжение"]